### Task:

Using PySpark transformations, merge data from two input files, 'employee.csv' and 'employee_personal.csv', to create an 'employee_fact' DataFrame. This DataFrame should include the following columns:

- `employee_id`
- `employee_full_name`
- `department`
- `salary`
- `Salary_Diff_to_reach_highest_sal`
- `DOB`
- `state`
- `country`
- `age`

Ensure that the 'employee_fact' DataFrame is structured to contain comprehensive information about each employee, including personal details such as full name, date of birth, state, and country, alongside professional details like department and salary.


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('challenges').getOrCreate()

In [18]:
employees_location = 'data/employee_data'

df_input_emp = spark.read.format('csv').options(inferSchema=True,header=True).load(fr"{employees_location}/employee.csv")

df_input_emp.show()

df_input_emp_per = spark.read.format('csv').options(inferSchema=True,header=True).load(fr"{employees_location}/employee_personal.csv")

df_input_emp_per.show()

+-----------+------------+-------+
|employee_id|  department| salary|
+-----------+------------+-------+
|          1|       Sales|50000.0|
|          2|   Marketing|60000.0|
|          3| Engineering|70000.0|
|          4|          HR|55000.0|
|          5|     Finance|65000.0|
+-----------+------------+-------+

+-----------+----------+---------+-------------------+-----------+-------+
|employee_id|first_name|last_name|                DOB|      state|country|
+-----------+----------+---------+-------------------+-----------+-------+
|          1|      John|      Doe|1980-05-15 00:00:00| California|    USA|
|          2|      Jane|    Smith|1985-09-20 00:00:00|   New York|    USA|
|          3|   Michael|  Johnson|1978-03-10 00:00:00|      Texas|    USA|
|          4|     Emily|    Brown|1990-11-25 00:00:00|    Florida|    USA|
|          5|     David|    Jones|1982-07-08 00:00:00|   Illinois|    USA|
+-----------+----------+---------+-------------------+-----------+-------+



In [19]:
df_join = df_input_emp.join(df_input_emp_per, on='employee_id', how = 'inner')

df_join.show()

+-----------+------------+-------+----------+---------+-------------------+-----------+-------+
|employee_id|  department| salary|first_name|last_name|                DOB|      state|country|
+-----------+------------+-------+----------+---------+-------------------+-----------+-------+
|          1|       Sales|50000.0|      John|      Doe|1980-05-15 00:00:00| California|    USA|
|          2|   Marketing|60000.0|      Jane|    Smith|1985-09-20 00:00:00|   New York|    USA|
|          3| Engineering|70000.0|   Michael|  Johnson|1978-03-10 00:00:00|      Texas|    USA|
|          4|          HR|55000.0|     Emily|    Brown|1990-11-25 00:00:00|    Florida|    USA|
|          5|     Finance|65000.0|     David|    Jones|1982-07-08 00:00:00|   Illinois|    USA|
+-----------+------------+-------+----------+---------+-------------------+-----------+-------+



In [26]:
df_join.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- DOB: timestamp (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)



In [44]:
from pyspark.sql.functions import *

highest_salary = df_join.select('salary').agg(max('salary')).rdd.map(lambda a: a[0]).collect()[0]

df = df_join.select(
    'employee_id', 
    concat(col('first_name'), lit(' '), col('last_name')).alias('full_name'),
    'department',
    'salary',
    (lit(highest_salary)-col('salary')).alias('Salary_Diff_to_reach_highest_sal'),
    'dob',
    'state',
    'country',
    round(datediff(current_timestamp(),col('dob'))/360,2).alias('Age')
)

df.show()


+-----------+-----------------+------------+-------+--------------------------------+-------------------+-----------+-------+-----+
|employee_id|        full_name|  department| salary|Salary_Diff_to_reach_highest_sal|                dob|      state|country|  Age|
+-----------+-----------------+------------+-------+--------------------------------+-------------------+-----------+-------+-----+
|          1|        John  Doe|       Sales|50000.0|                         20000.0|1980-05-15 00:00:00| California|    USA|44.61|
|          2|      Jane  Smith|   Marketing|60000.0|                         10000.0|1985-09-20 00:00:00|   New York|    USA|39.18|
|          3| Michael  Johnson| Engineering|70000.0|                             0.0|1978-03-10 00:00:00|      Texas|    USA|46.82|
|          4|     Emily  Brown|          HR|55000.0|                         15000.0|1990-11-25 00:00:00|    Florida|    USA|33.93|
|          5|     David  Jones|     Finance|65000.0|                        

In [41]:
highest_salary

[70000.0]